In [1]:
# Modifica las variables de WRF para poder hacer el mapeo de cuantiles.

import numpy as np
import pandas as pd

import xarray as xr
import os

# Inicializamos el dashboard de cómputo distribuido.
from dask.distributed import Client
c_lat = 1
c_lon = 1
Client( n_workers = 15, threads_per_worker = 5, memory_limit = "500MB" )

distributed.diskutils - INFO - Found stale lock file and directory '/Users/rodrigo/Documents/Posgrado/Doctorado/Code/recurso_solar/code/dask-worker-space/worker-7h0z3m27', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 15
Total threads: 75,Total memory: 6.98 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:50104,Workers: 15
Dashboard: http://127.0.0.1:8787/status,Total threads: 75
Started: Just now,Total memory: 6.98 GiB
Comm: tcp://127.0.0.1:50153,Total threads: 5
Dashboard: http://127.0.0.1:50154/status,Memory: 476.84 MiB
Nanny: tcp://127.0.0.1:50113,


In [2]:
# Cargamos el archivo.
path_d = "../data/WRF/"
path_r = "../temp/quantile_prep/"
path_v = "../temp/quantile_vars/"

# Iteramos para todos los archivos.
files = os.listdir(path_d)
files.sort()
if ".DS_Store" in files: files.remove(".DS_Store")
for f in files:
    print(f"{f[-5:-3]}", end=",")
    with xr.open_dataset( path_d + f, chunks = {
        "south_north": c_lat, "west_east": c_lon } ) as ds:

        # Creamos las variables.
        ds["Wind_Speed"] = np.sqrt( np.square(ds["U10"])
            + np.square(ds["V10"]) ).astype(np.float32)
        ds["Wind_Direction"] = ( np.arctan2(ds["V10"], ds["U10"])
            * 180/np.pi - 90 )
        ds["Wind_Direction"] = ds["Wind_Direction"].where(
            ds["Wind_Direction"] > 0, ds["Wind_Direction"] + 360
            ).astype(np.float32)
        ds = ds.drop_vars( ["U10", "V10"] )
        ds["T_C"] = ds["T2"] - 273.15
        ds["Pvs"] = 611.21*np.exp( ( 18.678 - ds["T_C"]/234.5 )
            * ( ds["T_C"]/(257.14+ds["T_C"]) ) )
        ds = ds.drop_vars( "T_C" )
        ds["Q2"] = ds["Q2"].where( ds["Q2"] > 0.0001, 0.0001 )
        ds["Relative_Humidity"] = ( 100 * ds["Q2"] * 461.4/286.9
            * ( ds["PSFC"]/ds["Pvs"] - 1 ) )
        ds["Relative_Humidity"] = ds["Relative_Humidity"].where(
            ds["Relative_Humidity"] < 100, 100 ).astype(np.float32)
        ds = ds.drop_vars( ["Q2", "Pvs"] )

        # Reordenamos el Dataset.
        ds["Wind_Speed"] = ds["Wind_Speed"].assign_attrs( units = "m/s" )
        ds["Relative_Humidity"] = ds[ "Relative_Humidity"
            ].assign_attrs( units = "%" )
        ds["Wind_Direction"] = ds ["Wind_Direction"
            ].assign_attrs( units = "degrees" )
        ds = ds.rename_vars( { "XTIME": "time", "XLONG": "lon", "XLAT": "lat",
            "T2": "Temperature", "PSFC": "Pressure", "SWDOWN": "GHI" } )
        ds["lon"] = ds["lon"].isel({"south_north": 0})
        ds["lat"] = ds["lat"].isel({"west_east": 0})
        ds = ds.swap_dims( {"south_north": "lat",
            "west_east": "lon", "XTIME": "time"} )

        # Guardamos el archivo.
        # Esta línea arranca el cómputo distribuido.
        ds.to_netcdf( path_r + f, mode = "w" )

        # Guardamos las variables individuales.
        vars = ["Temperature", "Pressure", "Relative_Humidity",
            "Wind_Speed", "Wind_Direction", "GHI"]
        for v in vars:
            ds[[v]].to_netcdf( path_v + v + "/" + f, mode = "w" )

00,